In [ ]:
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf

In [ ]:
import pathlib

data_train = './dataset/train'
data_test = './dataset/test'
data_val = './dataset/val'

In [ ]:
#preprocessing data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.ndimage import gaussian_filter

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode= "nearest",
        )

test_datagen = ImageDataGenerator(
    rescale=1./255,
    )

val_datagen = ImageDataGenerator(
    rescale= 1./255,
    )

train_generator = train_datagen.flow_from_directory(
    data_train,
    target_size=(224,224),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=44,
    shuffle = True
)

test_generator = test_datagen.flow_from_directory(
    data_test,
    target_size=(224,224),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=22,
    shuffle=False
)

val_generator = val_datagen.flow_from_directory(
    data_val,
    target_size=(224,224),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=11
)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=(3,3), strides=(1,1),padding="same",activation="relu", input_shape=(224,224,1)))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding="same",activation="relu"))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding="same",activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=256, kernel_size=(3,3),strides=(1,1), padding="same",activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding="same",activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(4, activation="softmax"))


In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
# create checkpoint and csvLogger
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
checkpoint_callback = ModelCheckpoint(filepath='./best_model.h5',
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      mode='max')

csvLogger = CSVLogger(filename="./loggingmodel.csv",separator=",", append=False)

In [ ]:
history = model.fit(
    train_generator,
    epochs = 100,
    steps_per_epoch= len(train_generator),
    validation_data = val_generator,
    verbose = 1,
    validation_steps=len(val_generator),
    callbacks=[checkpoint_callback, csvLogger]
)

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']


epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
acc = history.history['accuracy']

loss = history.history['loss']


epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
val_acc = history.history['val_accuracy']

loss = history.history['loss']


epochs = range(len(val_acc))

# plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training Validation Accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Loss Training')
plt.plot(epochs, val_loss, 'b', label='Loss Validation')
plt.title('Loss Accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, val_loss, 'b', label='Loss Validation')
plt.title('Loss Validation Accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
loss = history.history['loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Loss Training')
plt.title('Loss Accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

predictions = model.predict(test_generator)
predictions_labels = np.argmax(predictions, axis = 1)
true_labels = test_generator.classes

conf_matrix = confusion_matrix(true_labels, predictions_labels)


plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap="Blues", xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel("Predicted Labels")
plt.ylabel("True Label")
plt.title("Confussion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

class_report = classification_report(true_labels, predictions_labels, target_names=test_generator.class_indices.keys())
print(class_report)

In [ ]:
predictions_labels

In [ ]:
true_labels